# Сверточные нейронные сети

1. обучить рукописную сверточную нейронную сеть (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете cifar-10
2. оценить рост точности при увеличении ширины сети (больше фильтров)
3. оценить рост точности при увеличении глубины сети (больше слоев)
4. *сравнить с точностью полносвязной сети для этой выборки


In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
import tensorflow.keras as keras

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

tf.random.set_seed(1)

2023-09-23 00:01:35.460002: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-23 00:01:35.506720: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 00:01:35.788318: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 00:01:35.790063: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 00:01:36.646296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [3]:
# установка параметров нейросети
batch_size = 1000
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'тренировочные примеры')
print(X_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


X_train = X_train / 255.0
X_test = X_test / 255.0

X_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [27]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_24 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_25 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 15, 15, 32)        0         
                                                      

In [6]:
def train_augm(data_augmentation, model):
    if not data_augmentation:
        print('Не используется data augmentation')
        model.fit(X_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(X_test, y_test),
                  shuffle=True)
    else:
        print('Использование data augmentation')
        # Препроцессинг и data augmentation в реальном времени:
        datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            zca_epsilon=1e-06,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.,
            zoom_range=0.2,
            channel_shift_range=0.,
            fill_mode='nearest',
            cval=0.,
            horizontal_flip=True,
            vertical_flip=False,
            rescale=None,
            preprocessing_function=None,
            data_format=None,
            validation_split=0.0)


        train_gen = datagen.flow(X_train,
                                 y_train,
                                 batch_size=batch_size)

        # запуск data augmentation через fit_generator
        model.fit(train_gen,
                  epochs=epochs,
                  validation_data=(X_test, y_test))
    return model

In [29]:
model = train_augm(data_augmentation, model)

Использование data augmentation
Epoch 1/5
50/50 [==============================] - 29s 574ms/step - loss: 1.9542 - accuracy: 0.2771 - val_loss: 1.6532 - val_accuracy: 0.4026
Epoch 2/5
50/50 [==============================] - 35s 693ms/step - loss: 1.6359 - accuracy: 0.4042 - val_loss: 1.5555 - val_accuracy: 0.4338
Epoch 3/5
50/50 [==============================] - 35s 692ms/step - loss: 1.5253 - accuracy: 0.4492 - val_loss: 1.3157 - val_accuracy: 0.5250
Epoch 4/5
50/50 [==============================] - 35s 695ms/step - loss: 1.4366 - accuracy: 0.4855 - val_loss: 1.2608 - val_accuracy: 0.5502
Epoch 5/5
50/50 [==============================] - 35s 692ms/step - loss: 1.3710 - accuracy: 0.5119 - val_loss: 1.2031 - val_accuracy: 0.5730


In [30]:
# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print(f'сохранить обученную модель как {model_path}')

# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', round(scores[0],4))
print('Test accuracy:', round(scores[1], 4))

сохранить обученную модель как /home/ekaterina/Документы/Катя/ИИ/nn/lesson4/saved_models/keras_cifar10_trained_model.h5
313/313 [==============================] - 2s 5ms/step - loss: 1.2031 - accuracy: 0.5730
Test loss: 1.2031
Test accuracy: 0.573


## Создадим полносвязную сеть для данного датасета

In [31]:
model_1 = Sequential([
    Flatten(input_shape=(32, 32, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')
])

model_1.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )

model_1.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 3072)              0         
                                                                 
 dense_18 (Dense)            (None, 128)               393344    
                                                                 
 dense_19 (Dense)            (None, 64)                8256      
                                                                 
 dense_20 (Dense)            (None, 32)                2080      
                                                                 
 dense_21 (Dense)            (None, 10)                330       
                                                                 
Total params: 404010 (1.54 MB)
Trainable params: 404010 (1.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
model_1 = train_augm(data_augmentation, model_1)

Использование data augmentation
Epoch 1/5
50/50 [==============================] - 13s 246ms/step - loss: 2.1671 - accuracy: 0.2095 - val_loss: 2.0238 - val_accuracy: 0.2892
Epoch 2/5
50/50 [==============================] - 12s 239ms/step - loss: 1.9691 - accuracy: 0.2915 - val_loss: 1.8530 - val_accuracy: 0.3473
Epoch 3/5
50/50 [==============================] - 12s 237ms/step - loss: 1.8791 - accuracy: 0.3285 - val_loss: 1.8003 - val_accuracy: 0.3613
Epoch 4/5
50/50 [==============================] - 12s 239ms/step - loss: 1.8460 - accuracy: 0.3422 - val_loss: 1.7638 - val_accuracy: 0.3718
Epoch 5/5
50/50 [==============================] - 12s 238ms/step - loss: 1.8080 - accuracy: 0.3530 - val_loss: 1.7202 - val_accuracy: 0.3927


In [34]:
# проверка работы обученной модели
scores_1 = model_1.evaluate(X_test, y_test, verbose=1)
print('Test loss:', round(scores_1[0], 4))
print('Test accuracy:', round(scores_1[1],4))

313/313 [==============================] - 0s 882us/step - loss: 1.7202 - accuracy: 0.3927
Test loss: 1.7202
Test accuracy: 0.3927


Вывод: на данном этапе с использованием аугментации полносвязная сеть дает нам на 5 эпохах accuracy ниже, чем сверточная. Попробуем изменять параметры сверточной сети, чтобы еще улучшить результат.

In [35]:
# Изменим dropout на 0,15 на всех слоях, где он используется

batch_size = 1000
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model = train_augm(data_augmentation, model)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_30 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_21 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_31 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 15, 15, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 15, 15, 32)        0         
                                                      

In [36]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', round(scores[0],4))
print('Test accuracy:', round(scores[1], 4))

313/313 [==============================] - 2s 5ms/step - loss: 1.2054 - accuracy: 0.5721
Test loss: 1.2054
Test accuracy: 0.5721


Изменение dropout на 0,15 на всех слоях не улучшило модель, вернемся к предыдущему варианту.

In [41]:
# Изменим параметр количества экземпляров в слоях применения фильтров conv (16, 32, 64, 128 фильтров). 
# При этом padding везде сделаем "same". 

batch_size = 1000
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(16, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model = train_augm(data_augmentation, model)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 activation_48 (Activation)  (None, 32, 32, 16)        0         
                                                                 
 conv2d_33 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 activation_49 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 16, 16, 32)        0         
                                                     

In [42]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', round(scores[0],4))
print('Test accuracy:', round(scores[1], 4))

313/313 [==============================] - 2s 8ms/step - loss: 1.1239 - accuracy: 0.6060
Test loss: 1.1239
Test accuracy: 0.606


Результат значительно улучшился, далее работаем с последним вариантом

In [43]:
# Увеличим количество слоев. Добавим один слой conv + pooling
batch_size = 1000
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(16, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(200, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model = train_augm(data_augmentation, model)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 activation_54 (Activation)  (None, 32, 32, 16)        0         
                                                                 
 conv2d_37 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 activation_55 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_27 (Dropout)        (None, 16, 16, 32)        0         
                                                     

In [44]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', round(scores[0],4))
print('Test accuracy:', round(scores[1], 4))

313/313 [==============================] - 3s 9ms/step - loss: 1.2521 - accuracy: 0.5457
Test loss: 1.2521
Test accuracy: 0.5457


Результат ухудшился, берем предыдущий вариант без дополнительного conv+pooling

In [7]:
# Используем вариант с параметрами количества фильтров (16, 32, 64, 128 фильтров). 
# При этом padding оставляем сделаем "same". 
# Увеличим количество эпох до 10

batch_size = 1000
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(16, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model = train_augm(data_augmentation, model)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 32, 32, 16)        448       
                                                                 
 activation_12 (Activation)  (None, 32, 32, 16)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 activation_13 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 16, 16, 32)        0         
                                                      

In [8]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', round(scores[0],4))
print('Test accuracy:', round(scores[1], 4))

313/313 [==============================] - 2s 7ms/step - loss: 0.9640 - accuracy: 0.6575
Test loss: 0.964
Test accuracy: 0.6575


В результате увеличения количества эпох до 10 модель работает лучше.

In [9]:
# Используем вариант с параметрами количества фильтров (16, 32, 64, 128 фильтров). 
# При этом padding оставляем сделаем "same". 
# Количество эпох используем 10
# Добавим еще один полносвязный слой 256 нейронов

batch_size = 1000
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(16, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model = train_augm(data_augmentation, model)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 activation_18 (Activation)  (None, 32, 32, 16)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 activation_19 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 16, 16, 32)        0         
                                                      

In [10]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', round(scores[0],4))
print('Test accuracy:', round(scores[1], 4))

313/313 [==============================] - 2s 8ms/step - loss: 0.9927 - accuracy: 0.6527
Test loss: 0.9927
Test accuracy: 0.6527


Работа модели не улучшилась, поэтому добавлять полносвязный слой не будем.

In [12]:
# Используем вариант с параметрами количества фильтров (16, 32, 64, 128 фильтров). 
# При этом padding оставляем сделаем "same". 
# Увеличим количество эпох до 20

batch_size = 1000
num_classes = 10
epochs = 20
data_augmentation = True
num_predictions = 20

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(16, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model = train_augm(data_augmentation, model)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 activation_25 (Activation)  (None, 32, 32, 16)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 activation_26 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 16, 16, 32)        0         
                                                      

In [13]:
# проверка работы обученной модели
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', round(scores[0],4))
print('Test accuracy:', round(scores[1], 4))

313/313 [==============================] - 2s 7ms/step - loss: 0.7675 - accuracy: 0.7315
Test loss: 0.7675
Test accuracy: 0.7315


## Accuracy 0.7315  на датасете cifar-10 - это лучший результат из всех рассмотренных ранее вариантов

In [ ]:
# сохранение лучшей модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print(f'сохранить обученную модель как {model_path}')

### Вывод: наилучшего результата accuracy удалось добиться при модели с параметрами количества фильтров (16, 32, 64, 128 фильтров), padding = "same", количество эпох 20.

### Проверим качество полносвязной модели с таким же количеством эпох (20 эпох)

In [15]:
epochs = 20
model_1 = Sequential([
    Flatten(input_shape=(32, 32, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')
])

model_1.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )

model_1.summary()

model_1 = train_augm(data_augmentation, model_1)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 3072)              0         
                                                                 
 dense_11 (Dense)            (None, 128)               393344    
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 10)                330       
                                                                 
Total params: 404010 (1.54 MB)
Trainable params: 404010 (1.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Использование data

In [16]:
# проверка работы обученной модели
scores_1 = model_1.evaluate(X_test, y_test, verbose=1)
print('Test loss:', round(scores_1[0], 4))
print('Test accuracy:', round(scores_1[1],4))

313/313 [==============================] - 0s 902us/step - loss: 1.4861 - accuracy: 0.4728
Test loss: 1.4861
Test accuracy: 0.4728


 Полносвязная модель с тремя внутренними слоями (128, 64, 32 нейрона), выходным слоем 10 нейронов работает намного хуже, чем сверточная сеть с несколькими этапами фильтрации и уменьшения размерности данных и одним внутренним полносвязным слоем (512 нейронов), у обеих моделей количество эпох обучения 20, batch_size = 1000. У сверточной сети accuracy на тесте 0.7315, у полносвязной 0.4728. Вывод: для данной задачи целесообразно использовать сверточную сеть.

 Улучшило первоначальную модель изменение количества фильтров (итоговый вариант 16, 32, 64, 128 фильтров). padding сделали везде "same", увеличение количества эпох до 20.

 Ухудшало работу сверточной сети добавление еще одного полносвязного слоя 256 нейронов, изменение dropout на 0,15 на всех слоях, где он используется, добавление еще одного слоя фильтрации и уменьшения размерности conv + pooling.

 В дальнейшем при настройке модели можно дать рекомендацию попробовать еще увеличить количество эпох обучения, если позволят вычислительные мощности и поработать с batch-size, также можно попробовать применить фильтры другой размерности (мы использовали (3,3)).